### 1. Get Data

In [1]:
import os
import requests
import zipfile
from pathlib import Path

# Set up path for data
data_path = Path("going_modular/data/")
image_path = data_path / "pizza_steak_sushi"

# If the data path does not exist, create it
if image_path.is_dir():
  print(f"Directory {data_path} already exists")
else:
  print(f"Directory {data_path} does not exit, creating one...")
  data_path.mkdir(parents = True, exist_ok = True)

# Download pizza steak sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading pizza steak sushi data")
  f.write(request.content)

# Unzip pizza steak sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping pizza_steak_sushi.zip  ")
  zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Directory going_modular/data already exists
Unzipping pizza_steak_sushi.zip  


### 2. Create Datasets and DataLoaders (data_setup.py)

In [2]:
%%writefile going_modular/going_modular/data_setup.py

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir : str,
    test_dir : str,
    transform : transforms.Compose,
    batch_size : int,
    num_workers : int=NUM_WORKERS):

    """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """

    # Datasets from image folders
    train_data = datasets.ImageFolder(train_dir, transform = transform)
    test_data = datasets.ImageFolder(test_dir, transform = transform)

    # Get class names
    class_names = train_data.classes

    # Dataloaders from datasets
    train_dataloader = DataLoader(
        train_data,
        batch_size = batch_size,
        shuffle = True,
        num_workers = num_workers,
        pin_memory = True)

    test_dataloader = DataLoader(
        test_data,
        batch_size = batch_size,
        shuffle = False,
        num_workers = num_workers,
        pin_memory = True)

    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/going_modular/data_setup.py


In [3]:
from going_modular.going_modular import data_setup

### 3. Making a model(model_builder.py)

In [4]:
%%writefile going_modular/going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape : int, hidden_units : int, output_shape : int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(
            in_channels = input_shape,
            out_channels = hidden_units,
            kernel_size = 3,
            stride = 1,
            padding = 0
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels = hidden_units,
            out_channels = hidden_units,
            kernel_size = 3,
            stride = 1,
            padding = 0
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size = 3, padding = 0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size = 3, padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units*13*13,
                  out_features = output_shape)
    )

  def forward(self, x):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

Overwriting going_modular/going_modular/model_builder.py


In [5]:
import torch
from going_modular.going_modular import model_builder

device = "cuda" if torch.cuda.is_available() else "cpu"

class_names = ["pizza", "steak", "sushi"]
torch.manual_seed(42)
model = model_builder.TinyVGG(
    input_shape = 3,
    hidden_units = 10,
    output_shape = len(class_names)
).to(device)

### 4. Creating train_step() and test_step() functions and train() to combine them

In [6]:
%%writefile going_modular/going_modular/engine.py

import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model : torch.nn.Module,
               dataloader : torch.utils.data.DataLoader,
               loss_fn : torch.nn.Module,
               optimizer : torch.optim.Optimizer,
               device : torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_acc). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Train loss and accuracy
  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Forward pass
    y_pred = model(X)

    # Calculate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # Optimizer zero grad
    optimizer.zero_grad()

    # Backward propagation on the loss
    loss.backward()

    # Step the optimizer
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim = 1), dim = 1)
    train_acc += (y_pred_class == y).sum().item() / len(y_pred)

  train_loss /= len(dataloader)
  train_acc /= len(dataloader)
  return train_loss, train_acc

def test_step(model : torch.nn.Module,
              dataloader : torch.utils.data.DataLoader,
              loss_fn : torch.nn.Module,
              device : torch.device) -> Tuple[float, float]:

  # Put model on eval mode
  model.eval()

  # Test loss and accuracy
  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      # Forward pass
      y_pred_logits = model(X)

      # Calculate loss
      loss = loss_fn(y_pred_logits, y)
      test_loss += loss.item()

      # Calculate accuracy
      y_pred_label = torch.argmax(torch.softmax(y_pred_logits, dim = 1), dim = 1)
      test_acc += (y_pred_label == y).sum().item() / len(y_pred_logits)

  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)

  return test_loss, test_acc

def train(model : torch.nn.Module,
          train_dataloader : torch.utils.data.DataLoader,
          test_dataloader : torch.utils.data.DataLoader,
          optimizer : torch.optim.Optimizer,
          loss_fn : torch.nn.Module,
          epochs : int,
          device : torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty dictionary
  results = {"train_loss" : [],
             "train_acc" : [],
             "test_loss" : [],
             "test_acc" : []}

  for epoch in tqdm(range(epochs)):
    # Train model
    train_loss, train_acc = train_step(model = model,
                                       dataloader = train_dataloader,
                                       loss_fn = loss_fn,
                                       optimizer = optimizer,
                                       device = device)
    # Test model
    test_loss, test_acc = test_step(model = model,
                                    dataloader = test_dataloader,
                                    loss_fn = loss_fn,
                                    device = device)

    # Print what is happening
    print(f"Epoch: {epoch + 1}, train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}, test_loss: {test_loss:.4f}, test_acc: {test_acc:.4f}")

    # Update dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results

Overwriting going_modular/going_modular/engine.py


In [7]:
from going_modular.going_modular import engine

### 5. Creating a function to save the model (utils.py)

In [8]:
%%writefile going_modular/going_modular/utils.py

import torch
from pathlib import Path

def save_model(model : torch.nn.Module,
               target_dir : str,
               model_name : str):

  # Create target dictionary
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents = True,
                   exist_ok = True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith("pt")
  model_save_path = target_dir_path / model_name

  # Save the model state dict
  print(f"[INFO] saving model to {model_save_path}")
  torch.save(obj = model.state_dict(),
             f = model_save_path)

Overwriting going_modular/going_modular/utils.py


### 6. Train, evaluate and save the model (train.py)

In [48]:
#%%writefile going_modular/going_modular/train.py

import os
import torch
from going_modular.going_modular import data_setup, model_builder, engine, utils
from torchvision import transforms

# Setup hyper parameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "going_modular/data/pizza_steak_sushi/train"
test_dir =  "going_modular/data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
]
)

# Create dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    batch_size = BATCH_SIZE,
    transform = data_transform
)

# Create model
model = model_builder.TinyVGG(
    input_shape = 3,
    hidden_units = HIDDEN_UNITS,
    output_shape = 3
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

# Start training
engine.train(model = model,
             train_dataloader = train_dataloader,
             test_dataloader = test_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             epochs = NUM_EPOCHS,
             device = device)

# Save model
utils.save_model(model = model,
                 target_dir = "going_modular/models",
                 model_name = "05_going_modular_script_mode_tinyvgg_model.pth")


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1, train_loss: 1.0941, train_acc: 0.4336, test_loss: 1.0972, test_acc: 0.2604
Epoch: 2, train_loss: 1.0736, train_acc: 0.4258, test_loss: 1.1510, test_acc: 0.2604
Epoch: 3, train_loss: 1.1269, train_acc: 0.3047, test_loss: 1.1307, test_acc: 0.2604
Epoch: 4, train_loss: 1.1048, train_acc: 0.4023, test_loss: 1.1322, test_acc: 0.2396
Epoch: 5, train_loss: 1.0451, train_acc: 0.5352, test_loss: 1.0633, test_acc: 0.6042
[INFO] saving model to going_modular/models/05_going_modular_script_mode_tinyvgg_model.pth


### Exercises

#### 1. Turn the code to get the data (from section 1. Get Data above) into a Python script, such as get_data.py

In [11]:
#%%writefile going_modular/going_modular/get_data.py

import os
import requests
import zipfile
from pathlib import Path

# Set up path for data
data_path = Path("going_modular/data/")
image_path = data_path / "pizza_steak_sushi"

# If the data path does not exist, create it
if image_path.is_dir():
  print(f"Directory {data_path} already exists")
else:
  print(f"Directory {data_path} does not exit, creating one...")
  data_path.mkdir(parents = True, exist_ok = True)

# Download pizza steak sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading pizza steak sushi data")
  f.write(request.content)

# Unzip pizza steak sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping pizza_steak_sushi.zip  ")
  zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Directory going_modular/data already exists
Unzipping pizza_steak_sushi.zip  


In [12]:
!python3 going_modular/going_modular/get_data.py

Directory going_modular/data already exists
Unzipping pizza_steak_sushi.zip  


#### 2. Use Python's argparse module to be able to send the train.py custom hyperparameter values for training procedures.

In [28]:
%%writefile going_modular/going_modular/train.py

import os
import argparse
import torch
import data_setup, model_builder, engine, utils
from torchvision import transforms

# Create a parser
parser = argparse.ArgumentParser(description="Get some hyperparameters.")

# Get an argument for num epochs
parser.add_argument("--num_epochs",
                      default = 10,
                      type = int,
                      help = "the number of epochs to train for")

# Get an arg for batch_size
parser.add_argument("--batch_size",
                    default=32,
                    type=int,
                    help="number of samples per batch")

# Get an arg for hidden_units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help="number of hidden units in hidden layers")

# Get an arg for learning_rate
parser.add_argument("--learning_rate",
                    default=0.001,
                    type=float,
                    help="learning rate to use for model")

# Create an arg for training directory
parser.add_argument("--train_dir",
                    default="/content/going_modular/data/pizza_steak_sushi/train",
                    type=str,
                    help="directory file path to training data in standard image classification format")

# Create an arg for test directory
parser.add_argument("--test_dir",
                    default="/content/going_modular/data/pizza_steak_sushi/test",
                    type=str,
                    help="directory file path to testing data in standard image classification format")

# Get our arguments from the parser
args = parser.parse_args()

# Setup hyper parameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate

# Setup directories
train_dir = args.train_dir
test_dir = args.test_dir

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
]
)

# Create dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    batch_size = BATCH_SIZE,
    transform = data_transform
)

# Create model
model = model_builder.TinyVGG(
    input_shape = 3,
    hidden_units = HIDDEN_UNITS,
    output_shape = len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

# Start training
engine.train(model = model,
             train_dataloader = train_dataloader,
             test_dataloader = test_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             epochs = NUM_EPOCHS,
             device = device)

# Save model
utils.save_model(model = model,
                 target_dir = "going_modular/models",
                 model_name = "05_going_modular_script_mode_tinyvgg_model.pth")


Overwriting going_modular/going_modular/train.py


In [30]:
# Save model
utils.save_model(model = model,
                 target_dir = "going_modular/models",
                 model_name = "05_going_modular_script_mode_tinyvgg_model.pth")

[INFO] saving model to going_modular/models/05_going_modular_script_mode_tinyvgg_model.pth


In [29]:
!python3 going_modular/going_modular/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.00

  0% 0/5 [00:00<?, ?it/s]Epoch: 1, train_loss: 1.0990, train_acc: 0.3379, test_loss: 1.1004, test_acc: 0.2533
 20% 1/5 [00:31<02:05, 31.26s/it]Epoch: 2, train_loss: 1.0990, train_acc: 0.3342, test_loss: 1.1004, test_acc: 0.2533
 40% 2/5 [01:01<01:32, 30.92s/it]Epoch: 3, train_loss: 1.0991, train_acc: 0.3317, test_loss: 1.1004, test_acc: 0.2533
 60% 3/5 [01:32<01:01, 30.91s/it]Epoch: 4, train_loss: 1.0990, train_acc: 0.3354, test_loss: 1.1004, test_acc: 0.2533
 80% 4/5 [02:03<00:30, 30.78s/it]Epoch: 5, train_loss: 1.0990, train_acc: 0.3329, test_loss: 1.1004, test_acc: 0.2533
100% 5/5 [02:33<00:00, 30.78s/it]
Traceback (most recent call last):
  File "/content/going_modular/going_modular/train.py", line 99, in <module>
    utils.model_save(model = model,
AttributeError: module 'utils' has no attribute 'model_save'


### 3. Create a Python script to predict (such as predict.py) on a target image given a file path with a saved model.

In [66]:
%%writefile going_modular/going_modular/predict.py
import argparse
import torch
import model_builder
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
parser = argparse.ArgumentParser(description="Get an image path")

# Get an argument for num epochs
parser.add_argument("--image_path",
                      default = None,
                      type = str,
                      help = "The path to a single image")

# Get our arguments from the parser
args = parser.parse_args()
image_path = args.image_path

# Load the image
import torchvision
img = torchvision.io.read_image(image_path)


# Load the saved model
loaded_model = model_builder.TinyVGG(
    input_shape = 3,
    hidden_units = 10,
    output_shape = 3)
loaded_model.load_state_dict(torch.load(f="/content/going_modular/models/05_going_modular_script_mode_tinyvgg_model.pth"))
loaded_model = loaded_model.to(device)


# Make prediction on a sigle image
loaded_model.eval()
with torch.inference_mode():
  img = img / 255.

  resize = transforms.Resize((64, 64))
  img = resize(img)

  batch = img.unsqueeze(0).to(device)

  y_pred_logit = loaded_model(batch)

  pred_label = torch.argmax(y_pred_logit, dim=1)

class_names = ["pizza", "steak", "sushi"]
print(class_names[pred_label])


Overwriting going_modular/going_modular/predict.py


In [68]:
!python3 going_modular/going_modular/predict.py --image_path "/content/going_modular/data/pizza_steak_sushi/test/steak/100274.jpg"

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
steak
